* Data from https://opendata.dwd.de/climate_environment/GPCC/GPCC_DI/2023/GPCC_DI202307.nc

In [1]:
using YAXArrays,NetCDF

In [3]:
dataset=open_dataset("GPCC_DI_202307.nc",driver=:netcdf)

YAXArray Dataset
Shared Axes: 
Dim{:lon} Sampled{Float64} -179.5:1.0:179.5 ForwardOrdered Regular Points,
Dim{:lat} Sampled{Float64} -89.5:1.0:89.5 ForwardOrdered Regular Points,
Ti Sampled{Float64} Float64[0.0] ForwardOrdered Irregular Points
Variables: 
di_09, di_03, di_12, di_48, di_01, di_06, di_24, 
Properties: Dict{String, Any}("date_created" => "Thu Aug 10 04:18:46 UTC 2023", "creator_email" => "gpcc@dwd.de", "keywords" => "gpcc,global,gpcp,monthly,drought index,SPI,SPEI", "time_coverage_end" => "2023-07", "time_coverage_resolution" => "month", "time_coverage_start" => "2023-07", "history" => "Thu Aug 10 04:18:48 2023: cdo merge /media/x18914/GPCC_DI/TEMP/GPCC_DI_01_Month_202307.nc /media/x18914/GPCC_DI/TEMP/GPCC_DI_03_Month_202307.nc /media/x18914/GPCC_DI/TEMP/GPCC_DI_06_Month_202307.nc /media/x18914/GPCC_DI/TEMP/GPCC_DI_09_Month_202307.nc /media/x18914/GPCC_DI/TEMP/GPCC_DI_12_Month_202307.nc /media/x18914/GPCC_DI/TEMP/GPCC_DI_24_Month_202307.nc /media/x18914/GPCC_DI/TEMP/GPCC_

In [4]:
month_1=dataset["di_01"]
month_48=dataset["di_48"]

360×180×1 YAXArray{Union{Missing, Float32},3} with dimensions: 
  Dim{:lon} Sampled{Float64} -179.5:1.0:179.5 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float64} -89.5:1.0:89.5 ForwardOrdered Regular Points,
  Ti Sampled{Float64} Float64[0.0] ForwardOrdered Irregular Points
name: di_48
Total size: 253.12 KB


In [5]:
using GeoJSON
worldCountries = GeoJSON.read("../ne_110m_admin_0_countries.geojson")


FeatureCollection with 177 Features

In [11]:
using DimensionalData

In [15]:
lats=lookup(month_1,:lat)
lons=lookup(month_1,:lon)

Sampled{Float64} ForwardOrdered Regular Points
wrapping: -179.5:1.0:179.5

In [7]:
using GLMakie,GeoMakie

In [32]:
findfirst(lats.>40)


131

In [56]:
fig=Figure(resolution=(1200,1000))
Label(fig[1,1],text="The July 2023 in numbers, dry or wet?",fontsize=30)
Label(fig[2,1],text="Based on GPCC drought index for July 2023",fontsize=15)
ax=GeoAxis(fig[3,1],dest="+proj=eqearth +lon_0=35 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs",lonlims=(-20,65),latlims=(-40,40),title="Aggregation July 2023")
m_1=surface!(ax,lons[161:246],lats[51:131],month_1.data[161:246,51:131,1],colorrange=(-3.5,3.5),shading=false,colormap=cgrad(:vik,rev=true))
hm1=poly!(
    ax, worldCountries;
    color= :transparent,
    strokecolor = :black,
    strokewidth = 0.25)
translate!(hm1, 0, 0, 1000)
Colorbar(fig[3,2],vertical=true,m_1,ticks=(-3:1:3,["extremly dry","very dry","dry","normal","wet","very wet","extremly wet"]))


ax=GeoAxis(fig[4,1],dest="+proj=eqearth +lon_0=35 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs",lonlims=(-20,65),latlims=(-40,40),title="Aggregation last 48 months")
m_48=surface!(ax,lons[161:246],lats[51:131],month_48.data[161:246, 51:131,1],colorrange=(-3.5,3.5),shading=false,colormap=cgrad(:vik,rev=true))
hm1=poly!(
    ax, worldCountries;
    color= :transparent,
    strokecolor = :black,
    strokewidth = 0.25)
translate!(hm1, 0, 0, 1000)
Colorbar(fig[4,2],vertical=true,m_48,ticks=(-3:1:3,["extremly dry","very dry","dry","normal","wet","very wet","extremly wet"]))

Label(fig[5,1],text="Data: © Deutscher Wetterdienst\nMade with Natural Earth",justification=:left)
Makie.resize_to_layout!(fig)
fig

In [57]:
save("africa.png",fig)